# 1. Introduction
Variational bayes (VB) is a strategy for finding the optimal approximate posterior distribution under the Bayesian inference framework. Given latent variable $\mathbf{z}$ and observation $\mathbf{x}$, linked through the likelihood $p(\mathbf{x}|\mathbf{z})$, the posterior $p(\mathbf{z}|\mathbf{x})\propto p(\mathbf{x})p(\mathbf{x}|\mathbf{z})$ is a function of $\mathbf{z}$. For many problems we cannot compute $p(\mathbf{x})$ by integrating all possible values of the latent variable in reasonable amount of time. Then we need to approximate this. Rather than using stochastic methods such as MCMC to draw random samples from the posterior and then reconstruct the density, variational inference turns the approximation problem into an optimization problem by first choosing a family of approximate densities and then try to choose the density that minimizes the KL divergence to the exact posterior density. This works because at 0 KL divergence we uncover the exact posterior, and the smaller the divergence, typically, the better the match between our variational estimate to the true density.

# 2. Evidence Lower BOund (ELBO)

Now we formalize the optimization problem proposed in the previous section. Given a set of probability densities $Q=\{q_\theta(\mathbf{z})\}_{\theta\in \Theta}$ where $\Theta$ is the index set of variational parameters, we seek

$$
q^* = \underset{q\in Q}{\operatorname{argmin}} \operatorname{KL}(q\ ||\ p(\cdot|\mathbf{x})).
$$

We can rewrite the KL divergence as 
$$
\operatorname{KL}(q\ ||\ p(\cdot|\mathbf{x})) = \mathbb{E}_{\mathbf{z}\sim q(\mathbf{z})}[\log q(\mathbf{z})] - \mathbb{E}_{\mathbf{z}\sim q(\mathbf{z})}[\log p(\mathbf{z}, \mathbf{x})] + \log p(x).
$$

(Now we contract $\mathbb{E}_{\mathbf{z}\sim q(\mathbf{z})}$ to just $\mathbb{E}$, later we abbreviate taking expectation over a variational density as $\mathbb{E}_{q}$ if there will be no confusion in doing so) 

Note that $p(x)$ is what we can't quite compute, but we can rearrange to have 
$$
\log p(x) = \operatorname{KL}(q\ ||\ p(\cdot|\mathbf{x})) + (\mathbb{E}[\log p(\mathbf{z}, \mathbf{x})] - \mathbb{E}[\log q(\mathbf{z})])
$$

Since KL divergence is nonnegative, we have a lower bound for it
$$
ELBO(q)=\mathbb{E}[\log p(\mathbf{z}, \mathbf{x})] - \mathbb{E}[\log q(\mathbf{z})]=\int d\mathbf{z}\ q(\mathbf{z}) \log \frac{p(\mathbf{z}, \mathbf{x})}{q(\mathbf{z})}.
$$

It's clear that maximizing this is the same as minimizing the KL divergence because $p(\mathbb{x})$ is fixed. Now the question is how do we optimize this ELBO objective.

(Side note, the form of the above function is the same as the objective $F(q, \theta)$ for EM, explained in my other notebook!)

# 3. Mean-field variational family

(yes, this is very related to the mean-field approximation in statistical physics!)

We now have the burning question of how to compute the expectation with respect to the complete variational density over all latents, i.e., $\mathbb{E}_{\mathbf{z}\sim q(\mathbf{z})}$. Well, we start from a naive assumptions to simplify the problem. We assume that our variational density is factorizable over the latents:
$$
q(\mathbf{z}) = \prod_{j=1}^m q_j(z_j),
$$
which corresponds to a variational family $D=\times_{j=1}^m D_m$ where $D_m=\{q_{\theta_j}(z_j)\}_{j\in \Theta_j}$. This assumption may not always make sense, especially when there's clear covariation between some latents.

With this simplification we have
$$
\mathbb{E}[\cdot] = \mathbb{E}_{q_1}[\dots[\mathbb{E}_{q_m}[\cdot]]]
$$

and also
$$
\log p(\mathbf{z}, \mathbf{x}) = \log p(\mathbf{x}|\mathbf{z}) + \sum_{j=1}^m \log p(z_i)
$$
(I forgot where we used this)

# 4. Coordinate Ascent algorithm

The idea of coordinate ascent is simple. Given an objective, we iteratively choose a coordinate, fix all others, and move along this coordinate to optimize our function. Here in variational bayes, we can make use of the simplifying assumption introduced by the mean-field variational family to derive the optimum variational factor along each coordinate in the variational family. 

Based on this idea, we now write the ELBO as the function of variational factor $q_j$ only and leaves all other factors fixed. We use the observations from the last section:
$$
ELBO(q_j; q_{-j})=\mathbb{E}_j[\mathbb{E}_{-j}[p(z_j, \mathbf{z}_{-j}, \mathbf{x})]] - \mathbb{E}_j[\log q_j(z_j)]-\mathbb{E}_{-j}[\log q_{-j}(\mathbf{z}_{-j})].
$$

Note the last term does not depend on $q_j$, so we discard it when doing coordinate ascent along $j$-th factor's direction. But then we note the first two terms form a KL divergence:
$$
ELBO(q_j; q_{-j}) = \mathbb{E}_j\left[\log \frac{\exp(\mathbb{E}_{-j}[p(z_j, \mathbf{z}_{-j}, \mathbf{x})])}{q_j(z_j)}\right]
$$
which is minimized at $q^*_j = \exp(\mathbb{E}_{-j}[p(z_j, \mathbf{z}_{-j}, \mathbf{x})])$. 

So the coordinate ascent algorithm is really simple, conceptually. We start from some reasonable random parameters for each factor. While the ELBO has not converged yet, we go through $j=1,\dots, m$ and set $q_j$ to be what we found above, and then recompute ELBO to check convergence. We will be guaranteed to reach at a local maximum of the ELBO bound. The ELBO is typically nonconvex, so we should initialize from many different parameters for $q$. Note, the initialization affects the optimal factor found at each step through the expectation over all other variational factors.

# 5. Exponential families make life simple

This $q^*_j = \exp(\mathbb{E}_{-j}[p(z_j, \mathbf{z}_{-j}, \mathbf{x})]) \propto \exp(\mathbb{E}_{-j}[p(z_j| \mathbf{z}_{-j}, \mathbf{x})])$ is the heart of the CA algorithm from the previous section. We may ask when it could be made simpler. An natural exponential family (NEF) specifies a collection of conditional probability density $f(y|\theta)$ can be factored into three parts, one depend on $y$ alone, one dependent on $\theta$ alone and normalizes the distribution, and finally one that depends on $y$ and $\theta$ through $e^{y\theta}$. An exponential family generalizes by letting the last part be $e^{\theta T(y)}$, i.e., we have some sufficient statistics of the data as $T(y)$. Working with EF yields many distributions that are workhorses of modern statistics and ML, and these distributions have nice properties that can be proved for a general EF. We will not digress further. Now suppose we can write the complete conditional to be in the EF:
$$
p(z_j|\mathbf{z}_{-j}, \mathbf{x}) = h(z_j) \exp(\eta(\mathbf{z}_{-j}, \mathbf{x})\cdot z_j - a(\mathbf{z}_{-j}, \mathbf{x})),
$$

(Note that $\eta$ is the transform of conditioned variables into natural parameters of the EF, $a$ is the log normalizer, and $h$ is the underlying measure. Ignore if these jargons do not mean anything to you)

then we can rewrite the coordinate upate rule as
$$
q^*_j(z_j) \propto h(z_j) \exp(\mathbb{E}_{-j}[\eta(\mathbf{z}_j, \mathbf{x})]\cdot z_j)
$$

Note we have traded away the log normalizer to get a "propto" sign. Yet another simplification! This result looks great because it tells us the update will stay in the EF, but with $\eta$ replaced by $\mathbb{E}_{-j}[\eta(\mathbf{z}_j, \mathbf{x})]$.

# 6. Further model decomposition

Sometimes we can assume that the data is generated in two steps. Alternatively, we say there are two tiers of latent variables. First, we generate the global latent variables as a vector $\beta$ that control any part of the data. Next, we generate local latent variables as a vector whose components $\mathbf{z}$ define different contexts for generating the actual data $\mathbf{x}$.

The joint density will be
$$
p(\beta, \mathbf{z}, \mathbf{x})=p(\beta)\prod_i p(z_i,x_i|\beta).
$$

If the conditional is in EF:
$$
p(z_i,x_i|\beta) = h(z_i, x_i) \exp(t(z_i,x_i)\cdot \beta - a(\beta)),
$$

Clearly, we should model $p(\beta)$ to be the conjugate prior in the EF so that updating with each local context and data point will make the resulting distribution still in the EF. If we write
$$
p(\beta) = h'(\beta)\exp({\tau \beta - n_0 a(\beta) - a'(\tau, n_0)}). 
$$

which allows update of the natural parameters from $\tau, n_0$ to $\tau + \sum t(z_i, x_i), n_0 + n$.

Back to our variational bayes. Let $q(\beta)$ depend on global variational parameter $\lambda$, and each $q(z_i)$ depend on local variational parameter $\phi_i$. These variational parameters are defined with respect to our discussion about the exponential families above, which let the coordinate ascent updates change the natural parameters only. 

The joint density above reveals that the complete conditional of $z_i$ is independent of other $z_j$ and $x_j$'s where $j\neq i$. Thus the gradient ascent update will have the form 
$$
\phi_i^* = \mathbb{E}_\lambda [\eta(\beta, x_i)],
$$
and note that the expectation over other $\phi_j$'s are dropped because of conditional independence said above. We will iteratively perform all these updates of local parameters over all variational factors.

Finally, we need to deal with the global update. We note that fixing $x, z$, that $p(\beta|\mathbf{x},\mathbf{z})\propto p(\beta, \mathbf{x},\mathbf{z})$ which is in EF with natural parameters $[\tau + \sum t(z_i, x_i), n_0 + n]$ (we assume $t$ is the same throughout, i.e., we always use the same summary statistics). Hence, the global variational parameter will be given by 
$$
\lambda = E_{\phi_1,\dots,\phi_n}[[\tau + \sum t(z_i, x_i), n_0 + n]]=\left[\tau + \sum_i E_{\phi_i}[t(z_i, x_i)], \  n_0+n\right],
$$
where at the last step we used iterated expectation.

Finally...there's a huge long equation giving the ELBO in this case

# 7. Stochastic Variational Bayes

See Blei et al., not much new ideas here, just do gradient descent.

# 8. Gaussian mixture, revisited

A nice example to show how to use the EF family and the global-local decomposition as above to massively simplify the update rules. 

# 9. Final comment.

Expectation Maximization is a special case of variational bayes when the posterior variational family is restricted to delta functions (so to obtain point estimates, because MLE is equivalent to MAP with flat prior in Bayesian inference)